In [1]:
import getpass
import os
from langchain_openai import ChatOpenAI
if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [2]:
from datetime import datetime, timedelta
from typing import List, Dict, Optional
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from pydantic import Field
import json
import re

In [3]:
tasks = {"high":[],"medium":[],"low":[]}
reminders = {"high":[],"medium":[],"low":[]}

In [4]:
from typing_extensions import Annotated
from pydantic import BaseModel
class addTask(BaseModel):
    """Adds a new task with a deadline. Deadline can be in natural language like 'Monday' or 'tomorrow'."""
    title: Annotated[str,...,"Title of task."]
    deadline: Annotated[str,...,"Deadline of task."]
    priority:Annotated[str,...,"Priority of task.[High,Medium,Low]"]="medium"

class setReminder(BaseModel):
    """Sets a reminder for a specific task at a given time with priority (high, medium, low)."""
    task_title: Annotated[str,...,"Title of task."]
    reminder_time: Annotated[str,...,"Reminder Time."]
    priority:Annotated[str,...,"Priority of task.[High,Medium,Low]"]="medium"

class getQuery(BaseModel):
    type:Annotated[str,...,"task or reminder."]
    priority:Optional[Annotated[str,...,"priority of task[High,Medium,Low]"]]



In [6]:
@tool(args_schema=addTask)
def add_task(title: str, deadline: str, priority: str) -> Dict:
    """Adds a new task with a deadline. Deadline can be in natural language like 'Monday' or 'tomorrow'."""
    priority = priority.lower()
    if priority not in tasks:
        return {"error": "Invalid priority"}
    
    task = {
        "title": title,
        "deadline": deadline,
        "priority": priority,
        "created_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }
    tasks[priority].append(task)
    return task

# Set reminder tool
@tool(args_schema=setReminder)
def set_reminder(task_title: str, reminder_time: str, priority: str) -> Dict:
    """Sets a reminder for a specific task at a given time with priority (high, medium, low)."""
    priority = priority.lower()
    if priority not in reminders:
        return {"error": "Invalid priority"}
    
    reminder = {
        "task_title": task_title,
        "reminder_time": reminder_time,
        "priority": priority,
        "created_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }
    reminders[priority].append(reminder)
    return reminder

# Get query tool
@tool(args_schema=getQuery)
def get_query(type: str, priority: str = "") -> Dict:
    """Retrieves tasks or reminders based on the user's query. Type can be 'tasks' or 'reminders' and priority can be 'high', 'medium', or 'low'."""
    type = type.lower()
    priority = priority.lower()

    if type == "tasks":
        if priority in tasks:
            return {"tasks": tasks[priority]}
        return {"tasks": tasks}
    
    elif type == "reminders":
        if priority in reminders:
            return {"reminders": reminders[priority]}
        return {"reminders": reminders}
    
    return {"error": "Invalid query type or priority"}


In [7]:
from langchain_core.utils.function_calling import convert_to_openai_function
functions = [
    convert_to_openai_function(add_task),
    convert_to_openai_function(set_reminder),
    convert_to_openai_function(get_query)
]

In [8]:
functions

[{'name': 'add_task',
  'description': "Adds a new task with a deadline. Deadline can be in natural language like 'Monday' or 'tomorrow'.",
  'parameters': {'properties': {'title': {'type': 'string'},
    'deadline': {'type': 'string'},
    'priority': {'default': 'medium', 'type': 'string'}},
   'required': ['title', 'deadline'],
   'type': 'object'}},
 {'name': 'set_reminder',
  'description': 'Sets a reminder for a specific task at a given time with priority (high, medium, low).',
  'parameters': {'properties': {'task_title': {'type': 'string'},
    'reminder_time': {'type': 'string'},
    'priority': {'default': 'medium', 'type': 'string'}},
   'required': ['task_title', 'reminder_time'],
   'type': 'object'}},
 {'name': 'get_query',
  'description': "Retrieves tasks or reminders based on the user's query. Type can be 'tasks' or 'reminders' and priority can be 'high', 'medium', or 'low'.",
  'parameters': {'properties': {'type': {'type': 'string'},
    'priority': {'anyOf': [{'type

In [9]:
model = llm.bind(functions=functions)

In [11]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])

In [12]:
from langchain.schema.agent import AgentFinish
def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "add_task": add_task, 
            "set_reminder": set_reminder,
            "get_query":get_query
        }
        return tools[result.tool].run(result.tool_input)

In [13]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

In [14]:
result = chain.invoke({"input": "add task to cutting fuits on high priority for today."})

In [15]:
result

{'title': 'Cutting fruits',
 'deadline': 'today',
 'priority': 'high',
 'created_at': '2025-04-16 16:04:45'}

In [16]:
result = chain.invoke({"input": "which are available tasks for me on low priority today?"})

In [17]:
result

{'tasks': []}

In [18]:
result = chain.invoke({"input": "set eating sugar task on hi priority for today?"})

In [19]:
result = chain.invoke({"input": "add a reminder for driving car on high priority for tomorrow at 2 pm?"})

In [20]:
result

{'task_title': 'driving car',
 'reminder_time': 'tomorrow at 2 pm',
 'priority': 'high',
 'created_at': '2025-04-16 16:04:49'}

In [21]:
result = chain.invoke({"input": "set reminder for writing books on low priority on 4 pm?"})

In [22]:
result

{'task_title': 'writing books',
 'reminder_time': '4 pm',
 'priority': 'low',
 'created_at': '2025-04-16 16:04:50'}

In [23]:
result = chain.invoke({"input": "which are reminders for me on low priority?"})

In [24]:
result

{'reminders': [{'task_title': 'writing books',
   'reminder_time': '4 pm',
   'priority': 'low',
   'created_at': '2025-04-16 16:04:50'}]}

In [25]:
print(reminders)
print(tasks)

{'high': [{'task_title': 'driving car', 'reminder_time': 'tomorrow at 2 pm', 'priority': 'high', 'created_at': '2025-04-16 16:04:49'}], 'medium': [], 'low': [{'task_title': 'writing books', 'reminder_time': '4 pm', 'priority': 'low', 'created_at': '2025-04-16 16:04:50'}]}
{'high': [{'title': 'Cutting fruits', 'deadline': 'today', 'priority': 'high', 'created_at': '2025-04-16 16:04:45'}, {'title': 'Eating sugar', 'deadline': 'today', 'priority': 'high', 'created_at': '2025-04-16 16:04:47'}], 'medium': [], 'low': []}
